# Steganography Tool
#### By Ethan Sim

Steganography, according to CompTIA (https://www.comptia.org/blog/what-is-steganography), is the practice of hiding a secret message in a non-secret medium. What's the edge it gives us over cryptography? Consider a message like QFD FCGSG SFSEG - it's patently obvious that it's encoded! But what if we encountered a picture on a public website? Would we think to look for a message - even one written entirely in plaintext - there?

Here, we will implement a steganography technique known as Least Significant Bit encryption. Of course, we'll write a decryptor too! A more detailed overview can be found at https://towardsdatascience.com/hiding-data-in-an-image-image-steganography-using-python-e491b68b1372, but for those of us who want a quick summary, our strategy will be as follows:
- Convert an image's pixels to their RGB brightness values.
- Convert our message into binary - beginning and ending with a delimiter so we can decrypt it no matter where it is in the image.
- Replace the least significant bit of each pixel's colour's brightness values with that of our message.
- Test the image on our decryptor with a given delimiter.

The cool part of this technique is that you don't need to _read_ an image into binary to implement it - by replacing the least significant bit, you just add 1, subtract 1, or keep the number the same. The tricky part is iterating over all of these LSBs to find the message!

Consider a pixel with R = 99, which is 1100011 in binary. If our message had a 1 or 0 at this position, we'd either end up with 1100011 (returning us 99), or 1100010 (giving us 98) respectively.

## Image Conversion

First, we'll select an image and convert it to a RGB brightness representation. I've chosen this image of a dog by Wonderlane - it's in the public domain, so all's good. It's also 544 by 1023 pixels in size, so good luck finding a message inside!

<img src="cc_dog.jpg">

**Figure 1: Our chosen image. Source: https://search.creativecommons.org/photos/0542f64c-0063-46e2-844b-ce7fc2438078**

First things first, let's load our image in.

In [1]:
from PIL import Image
filename = "cc_dog.jpg"
img = Image.open(filename) #Now we can call the image using img!
#img.show() 
#This will display the image in an external window. 

Now, let's access the colour values of the image's constituent pixels!

In [2]:
pixel_map = img.load() #This is an array of pixels; each pixel is a tuple (R, G, B) where R, G, and B are numbers corresponding to the brightness values of each colour stream.
print (img.size) #This tells us how big the image is - 544 pixels wide and 1023 pixels long!
print (pixel_map[543, 1022]) #This is the bottom right corner

(544, 1023)
(231, 244, 255)


We only want to access the least significant bit of each pixel's colour stream. In this case, the least significant bits of the bottom right pixel are 1, 0, and 1 (Odd numbers have LSB = 1; even numbers have LSB = 0). By simply converting our colour streams into even or odd numbers (changing them by at most 1 to avoid making the modification too obvious), we can encode our message in the sequence of even and odd numbers! Having realised this, we can now take a break from our image and think about how we can convert our message to binary.

## Message Conversion

Our task here is simple (in theory) - Convert our message to binary. Whitespace has its own binary value, so we don't need individual character delimiters, but we do need message delimiters to tell our decryptor what to look out for later on. Arbitrarily, let's select ###### as a delimiter.

With this in mind, let's create a function to represent our message in binary format:

In [3]:
def message_to_binary(message, delimiter):
    """
    Takes in a string message, adds the string delimiter to both sides, converts all constituent characters to their binary representations, and returns a string binary_message.
    """
    message_copy = delimiter + message + delimiter #Add delimiters
    binary_message = ""
    for char in message_copy:
        binary_message += format(ord(char), '08b') #ord(char) gives char's ASCII code, and format(x, b) converts that number x into binary (eight-digit zero padded on the left).
        #Why use eight digits? Because our commonly used characters belong to the Basic Latin Unicode Block, i.e. their maximum binary value is 10000000. 
        #We'll decrypt them in blocks of eight later, which will spare us some time.
    return binary_message

In [4]:
print(message_to_binary("hello world", "######"))
print(len(message_to_binary("hello world", "######")))

0010001100100011001000110010001100100011001000110110100001100101011011000110110001101111001000000111011101101111011100100110110001100100001000110010001100100011001000110010001100100011
184


Observe how a message just 11 characters long (including whitespace) has now become 184 bits! With each pixel having 3 bits worth of encryption-space (and both delimiters taking up 96 bits in total), we could have a maximum of 544 * 1023 * 3 - 96 = 1669440 bits of information encoded in our chosen image. Given that we have 8 bits per character, we could encode a message at most 208680 characters long. Assuming that the average length of an English word is 5 characters (Bochkarev et al., 2012), we could take at most 41736 words - more than enough to smuggle out entire technical manuals!

Bochkarev et al., 2012: https://www.researchgate.net/publication/230764201_Average_word_length_dynamics_as_indicator_of_cultural_changes_in_society

## Hiding the Message

From the calculations above, we realise that there is a maximum message size any given image can take. If our image too small or message too large, we want to stop the user from attempting to encrypt. Otherwise, we'll simply provide a random starting point and encode accordingly, terminating before we run out of pixels.

NB: It's also extremely inconvenient for the user to type in their message in binary, so we'll wrap our encryption function around the message_to_binary function.

In [13]:
import math
import random

def hide_in_image(message, delimiter, image_filename, new_filename):
    """
    Takes in a string message, adds the string delimiter to both sides, and converts this to binary via message_to_binary.
    It then encrypts it within the image called image_filename, stored in the same folder as this workbook. 
    Saves the modified image as new_filename in the same folder.
    """
    #Sanity Checks
    if len(str(message)) == 0:
        return "Cannot encrypt without message!"   
    elif len(str(delimiter)) == 0:
        return "Cannot encrypt without delimiter!"
    
    binary_message = message_to_binary(str(message), str(delimiter)) #Message is now binary
    img = Image.open(image_filename)
    
    #Now, let's check the maximum message size
    max_size = img.size[0] * img.size[1] * 3
    if max_size < len(binary_message):
        return "Image too small or message too large!"
    
    #Set up the random starting point generator (not really random - we simply begin at the start of a randomly chosen column)
    pixels_used = math.ceil(len(binary_message) / 3.0) #Now we know how many pixels our message will take up
    #NB: We will run through our pixels columnwise, i.e. in the order (0,0), (0,1) ... (0, length), then (1,0), (1,1) ...
    columns_used = math.ceil(pixels_used / img.size[1]) #We divide by number of rows in a column
    #For example, if columns_used = 3, we must choose our starting column from the range 0 to ncol - 4, where the image has ncol columns of pixels.
    latest_start = img.size[0] - columns_used
    random_column = random.randrange(0, latest_start) #Choose random starting column - note that latest_start is excluded
    
    #Load pixel map
    pixel_map = img.load()
    
    #Refactoring dictionary
    evaluate_dict = {(0, 0): 0, (1, 1): 0, (0, 1): 1, (1, 0): -1} #This dictionary is of the format (LSB, message[idx]) : Modification to LSB
    
    #Begin encoding message
    message_idx = 0
    for pixel_col in range(random_column, random_column + columns_used):
    #for pixel_col in range(0, random_column + columns_used): #For testing purposes
        for pixel_row in range(0, img.size[1]):
            if message_idx == len(binary_message):
                break
            pixel = pixel_map[pixel_col, pixel_row] #This is a tuple with (R, G, B)
            new_value = [pixel[0], pixel[1], pixel[2]]
            for pixel_idx in range(0, 3):                
                evaluate_tuple = (pixel[pixel_idx] % 2, int(binary_message[message_idx]))
                new_value[pixel_idx] += evaluate_dict[evaluate_tuple] 
                message_idx += 1
                if message_idx == len(binary_message):
                    break
            pixel_map[pixel_col, pixel_row] = tuple(new_value) #This modifies the actual value
    
    #Message is now encoded, save the image
    img.save(new_filename + ".png") #We MUST use a lossless compression file format, like PNG - I tried this with a jpg for a few hours with no success. 
    print("Message encoded in " + new_filename + ".png!")

In [14]:
msg_string = "_Treatment._--If haemorrhage is still going on, it must be arrested by pressure, torsion, or ligature, as the accumulation of blood in a wound interferes with union. If necessary, the wound should be purified by washing with saline solution or eusol, and the surrounding skin painted with iodine, after which the edges are approximated by sutures. The raw surfaces must be brought into accurate apposition, care being taken that no inversion of the cutaneous surface takes place. In extensive and deep wounds, to ensure more complete closure and to prevent subsequent stretching of the scar, it is advisable to unite the different structures--muscles, fasciae, and subcutaneous tissue--by separate series of _buried sutures_ of catgut or other absorbable material. For the approximation of the skin edges, stitches of horse-hair, fishing-gut, or fine silk are the most appropriate. These _stitches of coaptation_ may be interrupted or continuous. In small superficial wounds on exposed parts, stitch marks may be avoided by approximating the edges with strips of gauze fixed in position by collodion, or by subcutaneous sutures of fine catgut. Where the skin is loose, as, for example, in the neck, on the limbs, or in the scrotum, the use of Michel's clips is advantageous in so far as these bring the deep surfaces of the skin into accurate apposition, are introduced with comparatively little pain, and leave only a slight mark if removed within forty-eight hours."
delimiter = "######"
hide_in_image(msg_string, delimiter, "cc_dog.jpg", "cc_dog_secret") 

Message encoded in cc_dog_secret.png!


At this stage, I have no idea if this has worked or not. We need to build a decryptor to determine what has happened.

## Retrieving the Message

Given our approach to encryption, we should inspect the start of every column to determine if a start delimiter is present. If so, we will convert the pixel values to binary and extract the message, stopping when we hit the end delimiter.

But first things first, let's build a function to read a message from binary.

In [15]:
def binary_to_message(binary):
    """
    Takes in a string binary encoded in 8-digit zero padded binary, decodes it, and returns a string decoded_message.
    """
    decoded_message = ""
    for binary_idx in range(0, len(binary), 8):
        decoded_message += chr(int("0b" + binary[binary_idx:binary_idx + 8], 2)) #0b is needed for Python's int() function to recognise a binary number
    return decoded_message

In [16]:
binary_to_message("0010001100100011001000110010001100100011001000110110100001100101011011000110110001101111001000000111011101101111011100100110110001100100001000110010001100100011001000110010001100100011")

'######hello world######'

So far, so good...

In [17]:
def reveal_message(image_filename, delimiter):
    """
    Assuming the message was encrypted using our schema, scans the image image_filename for encoded messages beginning and ending with the specified delimiter. 
    Returns a string final_message if a decoded message is present.
    """
    #Sanity Check
    if len(str(delimiter)) == 0:
        return "Cannot decrypt without a supplied delimiter!"
    
    img = Image.open(image_filename)
    
    #Load pixel map
    pixel_map = img.load()
    
    raw_binary = ""
    msg_found = False
    
    #We will use a brute force approach (to cover long delimiter edge cases) - but this also exploits elements of our strategy to save some time.
    #Essentially, after processing all our pixels into a long binary string, we will take slices at appropriate intervals and match them to the delimiter.
    for pixel_col in range(0, img.size[0]):
        for pixel_row in range(0, img.size[1]):
            pixel = pixel_map[pixel_col, pixel_row]
            for channel in pixel:
                raw_binary += str(channel % 2)
     
    delim_bin_length = len(str(delimiter)) * 8 #Assuming an 8-bit zero-padded approach
                            
    #Now we find the delimiter, if it exists
    for delim_start in range(0, len(raw_binary), img.size[1] * 3):
        prospective_delim = raw_binary[delim_start:delim_start + delim_bin_length] #This will cover long delimiters by simply reading them off the raw_binary string
        if binary_to_message(prospective_delim) == str(delimiter):
            msg_start = delim_start + delim_bin_length
            msg_found = True
            break

    if msg_found == False:
        return "No message found!"

    #By this point, we have identified the delimiter and can begin decoding
    preprocessed_message = binary_to_message(raw_binary[msg_start:])
    
    #Now we have a message in word form - but it has a whole lot of junk after the end delimiter - let's get rid of it
    final_message = preprocessed_message[:preprocessed_message.find(str(delimiter))]
        
    return final_message     

In [18]:
reveal_message("cc_dog_secret.png", delimiter)

"_Treatment._--If haemorrhage is still going on, it must be arrested by pressure, torsion, or ligature, as the accumulation of blood in a wound interferes with union. If necessary, the wound should be purified by washing with saline solution or eusol, and the surrounding skin painted with iodine, after which the edges are approximated by sutures. The raw surfaces must be brought into accurate apposition, care being taken that no inversion of the cutaneous surface takes place. In extensive and deep wounds, to ensure more complete closure and to prevent subsequent stretching of the scar, it is advisable to unite the different structures--muscles, fasciae, and subcutaneous tissue--by separate series of _buried sutures_ of catgut or other absorbable material. For the approximation of the skin edges, stitches of horse-hair, fishing-gut, or fine silk are the most appropriate. These _stitches of coaptation_ may be interrupted or continuous. In small superficial wounds on exposed parts, stitch

It works!

## Wrapping Up

Now all that's left is to build a simple interface...

In [19]:
def steg_user_interface():
    """
    This will allow the user to encrypt messages within a supplied image, or decrypt a message from a supplied image.
    """
    print("Would you like to encode? Y/N")
    encode_flag = str(input())
    if encode_flag == "Y":
        print ("Enter your message below.")
        message_string = str(input())
        print ("Enter your delimiter below.")
        encode_delimiter = str(input())
        print ("Enter the name of your image file below, e.g. 'cc_dog.jpg'.")
        image_name = str(input())
        print ("Enter the name you would like to call your modified file below, e.g.'cc_dog_secret'. Ignore all file extensions.")
        new_image_name = str(input())
        hide_in_image(message_string, encode_delimiter, image_name, new_image_name)
    elif encode_flag == "N":
        print("Would you like to decode? Y/N")
        decode_flag = str(input())
        if decode_flag == "Y":
            print ("Enter the name of your image file below, e.g. 'cc_dog_secret.png'")
            decode_image_name = str(input())
            print ("Enter your delimiter below.")
            decode_delimiter = str(input())
            print (reveal_message(decode_image_name, decode_delimiter))
        elif decode_flag == "N":
            print ("No worries, see you again!")
        else:
            print("Invalid input - Returning to start")
            steg_user_interface()
    else:
        print("Invalid input - Returning to start")
        steg_user_interface()
        
#Remember to keep pressing enter after you're done entering text!

In [12]:
#steg_user_interface()

#Uncomment this to begin!